# Eficiencia Julia

## Julia es rápido

Muy a menudo, técnicas de comparación o punt de referencia  (_benchmark_), son utilizadas para comparar lenguajes de programación. Estos puntos de referencia pueden dar lugar a largas discusiones, primero sobre qué se está evaluando exactamente y, en segundo lugar, qué explica las diferencias. Estas simples preguntas a veces pueden volverse más complicadas de que podemos imaginar al principio.

El propósito de los siguientes ejempos es mostrar un ejercicio de _benchmark_ sencillo y comprensible.

La versión original de este matrial nace como una maravillosa conferencia de Steven Johnson en el MIT: ["Boxes and registers"](https://github.com/stevengj/18S096/blob/master/lectures/lecture1/Boxes-and-registers.ipynb)

## Contenidos

- Definir la función `sum`
- Implementaciones y _comparativas_ de `sum`
    - Julia(built-in)
    - Julia(hand-writen)
    - C (hand-writen)
    - Python (built-in)
    - Python (numpy)
    - Python (hand-writen)
- Usando paralelismo con Julia
    - Utilización de la asociatividad de punto flotante
    - Utilización de 4 cores: built-in
    - Utilización de 4 cores: hand-writen
- Resumen de las comparativas


## `sum`: una función fácil de entender

Consideremos la función que suma elementos de un vector `sum(a)`, la cual se calcula:
$$sum(a) = \sum_{i=1}^n a_i$$ donde $n$ es el número de elementos (longitud) de $a$.

In [12]:
a = rand(10^7)    #vector 1D de números aleatorios uniformes [0,1)

10000000-element Vector{Float64}:
 0.24590583852547088
 0.35995467641472434
 0.6329305435576693
 0.2322304315824104
 0.20841559879792504
 0.07023987469640058
 0.04489961524560715
 0.8019785470934202
 0.27717248042455944
 0.880970697677699
 0.17222797545444624
 0.23423117855427655
 0.6189106535649207
 ⋮
 0.6941819403738738
 0.33278615527466426
 0.372253601666271
 0.19646111298770097
 0.45076281287266684
 0.6087669180968227
 0.44143563161586097
 0.09629563544784792
 0.1086215700073363
 0.12548021008969168
 0.20088355152368798
 0.8369571192322625

In [13]:
sum(a)

4.999780326291896e6

## Comparativas entre algunos cuantos lenguajes

In [14]:
@time sum(a)

  0.012632 seconds (1 allocation: 16 bytes)


4.999780326291896e6

In [ ]:
@time sum(a)

In [ ]:
@time sum(a)

La macro `@` puede devolver ciertos resultados sesgados, por tanto no es la mejor elección que hagamos. En su lugar utilizaremos el paquete `BenchmarkTools.jl` para hacer comparativas fáciles y mas precisas.

In [17]:
using Pkg
Pkg.add("BenchmarkTools")

using BenchmarkTools

   Resolving package versions...
   Installed BenchmarkTools ─ v1.1.1
    Updating `~/.julia/environments/v1.6/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.1.1
    Updating `~/.julia/environments/v1.6/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.1.1
  [2f01184e] + SparseArrays
  [10745b16] + Statistics
Precompiling project...
  ✓ BenchmarkTools
  1 dependency successfully precompiled in 3 seconds (17 already precompiled)


In [18]:
@benchmark sum($a)

BenchmarkTools.Trial: 513 samples with 1 evaluation.
 Range (min … max):  5.264 ms … 59.144 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     8.300 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   9.705 ms ±  5.365 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▄▇██▆▇▅                                                    
  ▆████████▆▆▅▅█▄▃▂▄▃▁▃▃▅▂▃▃▂▃▃▂▁▁▁▁▁▁▁▁▂▁▂▁▂▂▁▁▁▂▁▁▃▁▁▁▁▁▁▂ ▃
  5.26 ms        Histogram: frequency by time        34.3 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

## 1. Julia (built-in)

Bien, ya hemos visto el uso de la función predefinida `sum()`Julia. Por supuesto, está escrita en Julia, pero ¿funcionaría si escribiéramos una implementación nosotros mismos?

In [19]:
@which sum(a)

sum(a::AbstractArray; dims, kw...) in Base at reducedim.jl:873

Guardemos estos resultados de referencia en un diccionario para que podamos comenzar a realizar un seguimiento de ellos y compararlos en el futuro.

In [20]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 646 samples with 1 evaluation.
 Range (min … max):  5.219 ms … 17.220 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     7.470 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.704 ms ±  1.518 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

         ▄▃▂▂▂▅▂▃▄▄█▄▂▁ ▂▂ ▁                                  
  ▄▃▃▄▇▆▇█████████████████▇██▆█▆▅▅▆▄▅▄▄▄▃▂▃▁▃▃▁▃▂▁▃▂▃▁▁▂▁▃▂▃ ▄
  5.22 ms        Histogram: frequency by time        12.6 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [21]:
j_bench.times

646-element Vector{Float64}:
 5.218543e6
 5.241885e6
 5.258058e6
 5.2758e6
 5.344341e6
 5.351261e6
 5.416943e6
 5.439698e6
 5.507786e6
 5.510004e6
 5.531041e6
 5.652569e6
 5.670825e6
 ⋮
 1.2113211e7
 1.2263123e7
 1.2293862e7
 1.2394168e7
 1.2579062e7
 1.263212e7
 1.4514545e7
 1.4735268e7
 1.5171051e7
 1.5333928e7
 1.6897435e7
 1.7220006e7

In [22]:
d = Dict()
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

Dict{Any, Any} with 1 entry:
  "Julia built-in" => 5.21854

## 2. Julia (hand-written)

In [8]:
function mysum(A)
    s = 0.0
    for a in A
        s += a
    end
    return s
end

mysum (generic function with 1 method)

In [9]:
mysum(a)

LoadError: UndefVarError: a not defined

In [ ]:
j_bench_hand = @benchmark mysum($a)

In [ ]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

## 3. Lenguaje C

C a menudo se considera el estándar de oro: difícil para el ser humano, agradable para la máquina. Lograr estar dentro de un factor de ~2xC a menudo es satisfactorio. No obstante, incluso dentro de C, hay muchos tipos de optimizaciones posibles de las que un desarrollador de C novato puede o no aprovechar.

El autor actual no habla C, por lo que no comprende la celda de abajo, pero está feliz de saber que puede poner código C en una sesión de Julia, compilarlo y ejecutarlo. Tengamos en cuenta que `"""` envuelve una cadena de varias líneas.

In [23]:
using Libdl
C_code = """
    #include <stddef.h>
    double c_sum(size_t n, double *X) {
        double s = 0.0;
        for (size_t i = 0; i < n; ++i) {
            s += X[i];
        }
        return s;
    }
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code)
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [24]:
c_sum(a)

4.999780326291078e6

In [25]:
c_bench = @benchmark c_sum($a)

BenchmarkTools.Trial: 365 samples with 1 evaluation.
 Range (min … max):  10.969 ms … 38.320 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     12.967 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   13.651 ms ±  2.609 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ▃▆▅▃█▃ ▂  ▃ ▁                                             
  ▄▄▆███████████▇█▇█▅▅▄▄▆▅▃▄▃▂▃▄▁▂▃▃▁▂▃▂▃▁▁▂▁▃▁▁▂▁▂▂▃▂▁▁▁▁▂▁▂ ▃
  11 ms           Histogram: frequency by time        22.1 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [26]:
d["C"] = minimum(c_bench.times) / 1e6  # in milliseconds
d

Dict{Any, Any} with 2 entries:
  "C"              => 10.9687
  "Julia built-in" => 5.21854

## 4. Python (built in `sum`)

El paquete `PyCall` provee una interfase en Julia para usar Python

In [27]:
import Pkg; Pkg.add("PyCall");
using PyCall

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`


In [28]:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [29]:
pysum(a)

4.999780326291078e6

In [30]:
py_bench = @benchmark $pysum($a)

BenchmarkTools.Trial: 4 samples with 1 evaluation.
 Range (min … max):  1.485 s …   1.560 s  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.552 s              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.537 s ± 35.459 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁                                            ▁          █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁█ ▁
  1.48 s         Histogram: frequency by time        1.56 s <

 Memory estimate: 336 bytes, allocs estimate: 6.

In [31]:
d["Python built.in"] = minimum(py_bench.times) / 1e6
d

Dict{Any, Any} with 3 entries:
  "C"               => 10.9687
  "Python built.in" => 1484.96
  "Julia built-in"  => 5.21854

## 5. Python Numpy

`Numpy` es una biblioteca en C optimizada y se llama directamente desde Python:

In [32]:
using Pkg
Pkg.add("Conda")
using Conda

   Resolving package versions...
    Updating `~/.julia/environments/v1.6/Project.toml`
  [8f4d0f93] + Conda v1.5.2
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`


In [33]:
Conda.add("numpy")

┌ Info: Running `conda install -y numpy` in root environment
└ @ Conda /Users/antonioayala/.julia/packages/Conda/sNGum/src/Conda.jl:128


Solving environment: ...working... done

# All requested packages already installed.



In [34]:
numpy_sum = pyimport("numpy")["sum"]

PyObject <function sum at 0x7fcdef4dd820>

In [35]:
py_numpy_bench = @benchmark $numpy_sum($a)

BenchmarkTools.Trial: 602 samples with 1 evaluation.
 Range (min … max):  5.615 ms … 62.131 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     7.885 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   8.264 ms ±  2.897 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▁   ▃▂▃▅▅▃▆▇█▇▅▂▃█ ▁▂                                     
  ▄▅███▇██████████████████▆█▅▅▇▅▅▄▆▁▄▃▃▂▂▂▃▂▃▂▂▂▂▁▂▂▂▂▁▁▂▁▂▃ ▅
  5.62 ms        Histogram: frequency by time        14.3 ms <

 Memory estimate: 336 bytes, allocs estimate: 6.

In [36]:
numpy_sum(a)

4.999780326291898e6

In [38]:
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6
d

Dict{Any, Any} with 4 entries:
  "C"               => 10.9687
  "Python numpy"    => 5.61531
  "Python built.in" => 1484.96
  "Julia built-in"  => 5.21854

# 6. Python (hand-written)

In [39]:
py"""
def py_sum(A):
    s = 0.0
    for a in A:
        s += a
    return s
"""

sum_py = py"py_sum"

PyObject <function py_sum at 0x7fce03e9e0d0>

In [40]:
py_hand = @benchmark $sum_py($a)

BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  2.037 s …    2.382 s  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.220 s               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.213 s ± 172.528 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                             █                          █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  240 s          Histogram: frequency by time         2.38 s <

 Memory estimate: 336 bytes, allocs estimate: 6.

In [41]:
sum_py(a)

4.999780326291078e6

In [42]:
d["Python hand-written"] = minimum(py_hand.times) / 1e6
d

Dict{Any, Any} with 5 entries:
  "C"                   => 10.9687
  "Python numpy"        => 5.61531
  "Python built.in"     => 1484.96
  "Python hand-written" => 2037.38
  "Julia built-in"      => 5.21854

## Resumen

In [ ]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 25, "."), lpad(round(value; digits=1), 6, "."))
end

**Ejercicio:**
Implementar la multiplicación matriz-vector $M \times V$ donde $M$ es una matriz y $V$ un vector, ambos con las dimensiones adecuadas. Envolver el cálculo en una función `multmatvec` y calcular los tiempos de ejecución con `@benchmark`.